In [1]:
import numpy as np                                                              # Linear Algebra

import pandas as pd                                                             # Data Processing
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import datetime as dt                                                           # DateTime Manipulation

# Scatter 

In [2]:
PATH = r"C:\\Users\\btros\\OneDrive\\Documents\\LSDS-Unit-2-Project\\raw_csvs"

file = r"\\911_Calls_for_Service.csv"

df = pd.read_csv(PATH+file, low_memory = False)

In [3]:
def reduce_features(df):
    cols = ['X', 'Y', 'incident_id', 'oid']
    
    df = df.drop(columns=cols)
    
    return df

def drop_null_cols(df):
    
    columns_with_null = df.isnull().sum().sort_values(ascending=False)[:9].index.tolist()
    df = df.drop(columns=columns_with_null)
    
    return df

def extract_timestamp_feats(df, column):
    
    # Cast the column to DateTime type
    df[column] = pd.to_datetime(df[column])
    
    # Extract date related features
    df['call_year']  = df[column].dt.year
    df['call_month'] = df[column].dt.month
    df['call_day']   = df[column].dt.day
    df['call_dow']   = df[column].dt.dayofweek
    df['call_week']  = df[column].dt.week
    
    # Extract time related features
    df['call_hour']   = df[column].dt.hour
    df['call_minute'] = df[column].dt.minute
    """
    Extacts the time of day from the hour value by dividing by the hour knife
    1 = Morning  (0400 - 1000h)
    2 = Midday   (1000 - 1600h)
    3 = Evening  (1600 - 2200h)
    4 = Night    (2200 - 0400h)
    """    
    hour_knife = 6
    df['part_of_day'] = ((df['call_hour'] + 2) / hour_knife).astype(int)
    df['part_of_day'] = df['part_of_day'].replace(0, 4)
    
    # Drop the now redundant column
    df = df.drop(columns=column)
    
    return df

def filter_geo_outliers(df):
    """
    Filters out the outliers in the data where the latitude and longitudes 
    exist outside the geographic limits of the city of Detroit
    """    
    n_bound =  42.46
    e_bound = -82.91
    s_bound =  42.25
    w_bound = -83.28
    
    #  Retains only those observations within the geographic boundaries
    df = df.loc[((df['longitude'] > w_bound) & (df['longitude'] < e_bound)) &
                ((df['latitude'] > s_bound) & (df['latitude'] < n_bound ))]
    
    return df 

def remove_categoricals(df):
    unneeded = df.select_dtypes(exclude="number").nunique().index.tolist()
    unneeded.remove('calldescription')

    df = df.drop(columns=unneeded)
    
    return df

def clean_observations(df):    
    """
    Removing observations where the call description
    is not related to emergency services
    """
    col = 'calldescription'
    
    unrelated_calls = ['START OF SHIFT INFORMATION', 'CALL BACK DESK', 'REMARKS',
                      'EMPLOYEE CALL IN / TIME OFF', 'NOTIFICATION(S) MADE',
                      'TIBURON HISTORICAL DATE', 'BLUE LIGHT PHONE MALF']
    
    for call in unrelated_calls:
        df = df.loc[df[col] != call]
    
    return df

def filter_calldescription(df, col='calldescription'):
    
    #  List of call descriptions according to which service responded
    
    police = ['TRAFFIC STOP', 'SPECIAL ATTENTION', 'DISTURBANCE', 'INVESTIGATE PERSON',
              'ASSAULT AND BATTERY', 'UNKNOWN PROBLEM', 'TOWING DETAIL', 'FELONIOUS ASSAULT IP',
              'LARCENY REPORT', 'AUTO X UNK INJ / IMPAIRED', 'BUS BOARDING', 'PERSON WITH WEAPON',
              'VERIFIED ALR / PERSON W/O CODE', 'UDAA REPORT', 'BURGLARY OCCUPIED RESD I/P',
              'SHOTS FIRED IP', 'INVESTIGATE AUTO', 'MALICIOUS DESTRUCTION I/P', 'MISCELLANEOUS TRAFFIC',
              'PANIC / DURESS ALARM', 'FRAUD REPORT', 'BUILDING CHECK', 'VICIOUS ANIMAL', 'LARCENY I/P OR J/H',
              'THREATS REPORT', 'RECOVER AUTO', 'HOLD UP ALARM AND MOW', 'MALICIOUS DESTRUCTION RPT', 
              'BURGLARY OTHER', 'PARKING COMPLAINT', 'WELL BEING CHECK', 'MENTAL VIOLENT NOT ARMED',
              'ASSIST OTHER', 'INFORMATION/NON-CRIMINAL RPT', 'ASSAULT AND BATTERY REPORT', 'AUTO X REPORT',
              'BURGLARY OCCUPANT AWAY I/P', 'MISSING REPORT',  'NARCOTICS I/P', 'BURGLARY RESIDENCE REPORT',
              'MISSING SERIOUS', 'ALARM UNKNOWN CAUSE', 'SUICIDE THREAT', 'CHILD /  ADULT ABUSE',  'WNTD WRRNT FELONY',
              'ADMIT OR E/E', 'FELONIOUS ASSAULT JH', 'ASSIST CITIZEN', 'SPECIAL ATTENTION / DETAIL', 'DEAD PERSON OBSERVED',
              'LOST PROPERTY', 'SUICIDE I/P', 'TRANSPORT PRISONER', 'ROBBERY ARMED IP-ANY', 'RECOVERED / FOUND PROPERTY',
              'SAFEWALK', 'CHILD / ADULT ABUSE REPORT', 'BREAKING AND ENTERING AUTO I/P', 'FELONIOUS ASSAULT REPORT',
              'RAID - EXECUTE SEARCH WARRANT', 'MENTAL VIOLENT - ARMED', 'RAPE REPORT', 'HARASSMENT REPORT',
              'BURGLARY BUSINESS IP', 'KIDNAPPING', 'PPO VIOLATION I/P', 'LEWD AND LASCIVIOUS IP', 'AID MOTORIST / CHILD LOCKED IN',
              'BREAKING & ENTERING AUTO RPT', 'DV A/B I/P-J/H', 'ROBBERY NOT ARMED I/P', 'SQUATTER DISTURBANCE',
              'SCRAP STRIP  BLDG  IP', 'ASSAULT AND BATTERY I/P', 'AUTO OR PED H&R REPORT', 'FRAUD I/P',
              'AUTO X HIT & RUN W/ INJ', 'BURGLARY OCCUP RESD REPT', 'RUBBISH LITTERING I/P', 'SHOTS J/H, EVIDENCE, REPT',
              'BE ON THE LOOK OUT', 'SHOOTING/CUTTING/PENT WND RPT', 'HOLDING PERSON', 'ARSON I/P', 'BURGLARY OTHER REPORT',
              'ANIMAL COMPLAINT', 'ARSON REPORT', 'ASSIST PERSONNEL', 'ROBBERY ARMED REPORT', 'VERIFY RETURN OF MISSING',
              'DDOT TROUBLE', 'WNTD WRRNT MISDEMEANOR', 'ROBBERY ARMED JH-ANY WEAPON',  'BURGLARY OCCUPANT AWAY J/H',
              'UDAA I/P', 'SCHOOL CROSSING', 'HAZARDOUS CONDITIONS', 'SENIOR CITIZEN ASSIST', 'ROBBERY NOT ARMED REPORT',
              'ONE OVER THE WHEEL', 'RAPE IP OR JH', 'PROPERTY DAMAGE NON-CRIMINAL', 'PEACE OFFICER DETAIL', 'BURGLARY BUSINESS REPORT',
              'BURGLARY BUSINESS J/H', 'CHILD(REN) HOME ALONE', 'DV F/A I/P-J/H', 'ESCORT', 'NARCOTICS REPORT', 'AID MOTORIST MISC',
              'PPO VIOLATION REPORT', 'RECOVER AUTO FELONY', 'LEWD AND LASCIVIOUS','VEHICLE FIRE', 'NOISE COMPLAINT', 
              'FOUND PERSON', 'MOLESTATION REPORT', 'PBT TEST', 'RESIDENTIAL STRUCTURE FIRE', 'ASSIST REF GAINING ENTRY',
              'ATM  ALARM',  'WSPD - BURGLARY ALR', 'ALTERED LOC OR UNK PROBLEM', 'PAST GUNSHOT OR STABBINIG', 'AID MORTORIST MISC',
              'BOMB THREAT REPORT',  'ASSAULT  NOT DANGEROUS OR PREV', 'WRKABLE ARRST/OBV OR EXP DEATH',
              'CITIZEN RADIO PATROL IN TROUBL', 'SCHOOL THREATS J/H & RPT', 'TRAFFIC W/ NO INJURIES',  'MISCELLANEOUS ACCIDENT',
              'MOLESTATION', 'BANK ALARM','KIDNAPPING REPORT', 'K-9 DEPLOYMENT', 'SUSPICIOUS PACKAGE', 'DPDA', 
              'WSPD - BURGLARY ALARM', 'VIP THREATS I/P', 'PERSON W/  A WEAPON REPORT', 'AID MOTORIST CHILD LOCKED INSD',
              'ANIMAL FIGHT', 'BOMB THREAT', 'PERSONNEL IN TROUBLE', 'BURGLARY ALARM', 'DV PRS WITH A WEAP I/P-J/H',
              'ASSAULT AND BATTERY J/H', 'AUTO X - BLDG / DWELL', 'PURSUIT - VEHICLE OR FOOT', 'BACKGROUND/LEIN CHK / LIVESCAN',
              'INVESTIGATE YOUTH(S)', 'SHOT STABBED UNK OR OBV DEATH', 'SCRAP STRIP BLDG JH', 'TRANSPORT PRISONER-OTH AGT', 'BOMB SCARE',
              'SMOKING VIOLATIONS', 'DV A/B RPT', 'HPPD BURG ALRM', 'EXTORTION IP', 'EXTORTION JH OR REPORT',
              'OTHR OUTSIDE STRUCTURE FIRE', 'ATTEMPTED SUICIDE REPORT', 'VIP THREATS J/H OR REPORT', 'ASSAULT DANGEROUSOR SERIOUS',
              'TRAFFIC- MINOR INJURIES', 'SUICIDE THREAT OR ABNORM BEHAV', 'BURGLARY ALARM  W/ MOW', 'HPPD BURG ALRM  W/ MOW',
              'BACKGROUND CHECK / LIVESCAN', 'DV F/A RPT', 'CURFEW VIOLATION', 'YOUTH LOITERING/CONGREGATING']

    ems = ['ONE DOWN OR OVER THE WHEEL', 'AUTO X OR PED X - INJURIES', 'AUTO X HIT& RUN UNK INJURIES',
           'MENTAL VIOLENT NOT ARMED', 'MENTAL NOT VIOLENT', 'SHOOTING/CUTTING/PENT WOUND', 'MT EMS-TRO/ENTRY',
           'ONE DOWN/DRUG OD', 'ANIMAL BITES OR ATTACK DELTA', 'SHOT OR STABBED DELTA', 'PREGNANCY OR CHILDBIRTH DELTA',
           'CHEST PAIN DELTA', 'ATYPICAL SEIZURE OR HX OR CVA', 'OB HEMORRHAGE LABOR OR BIRTH',
           'CHEST PAIN NORMAL BREATHING', 'INJURY NON DANGEROUS BODY PART', 'NON SUICIDAL OR NT THRTENING',
           'SICK PERSON DELTA', 'ANIMAL BITE OR ATTACK DELTA', 'FALL W SERIOUS HEMORRHAGE', 'CONT OR MULTI SEIZURES DELTA',
           'PORTABLE ALARM SYSTEM', 'FAINTING ALERT', 'SERIOUS HEMORRHAGE', 'BURNS DELTA  OR ECHO', 'HEADACHE - ADDITIONAL SYMPTONS',
           'NON RECENT OR SUPERFICIAL BITE', 'ASSAULT OR SEX ASSAULT DELTA', 'UNCONSCIOUS OR FAINTING DELTA', 'DV PRSN WITH WEAP RPT',
           'SCHOOL THREATS I/P', 'SICK NON PRIORITY COMPLAINTS', 'SUICIDE ATTEMPT DELTA', 'DANGEROUS HEMORRHAGE DELTA',
           'SERIOUS INJURIES', 'MINOR HEMORRHAGE', 'BREATHING PROBLEMS DELTA', 'CHEST PAIN W OR WO NOR BRETH', 'OVERDOSE DELTA',
           'OVER THE WHEEL',  'TRAFFIC INCIDENT W/ INJURIES', 'POSS DANGEROUS ANIMAL BITE', 'ONE DOWN/OVERDOSE',
           'MAJOR TRAFFIC INCIDENT DELTA', 'OVERDOSE NT ALRT OR UNK STATUS', 'MEDICAL ALARM OR UNK PROB','LIFE STATUS QUESTIONABLE DELTA']

    fire = ['FIRE ALARM TEST', 'FIRE ALARM', 'TEMPERATURE ALARM',  'ELEVATOR ENTRAPMENT', 'VEHICLE FIRE(FIELD)',
            'IRRADIATOR ROOM ALARM / TEST', 'HAZARD CONDITIONS NO FIRE', 'GRASS  FIRE', 'POWER LINES', 'ALR PT DISABLED / TIMEZONE CHG',
            'EXPLOSION/RUPTURE NO FIRE', 'ALARM MALFUNCTION',  'COMMERCIAL STRUCTURE FIRE', 'EXPLOSION',  'FIRE ALARMS ALL',
            'WEATHER RELATED', 'GRASS OR RUBBISH FIRE',  'SMOKE INVESTIGATION', 'WSPD - FIRES', 'ALARM MISUSE']
    
           
    df['police'] = np.where(df[col].isin(police), 1, 0)
    df['ems'] = np.where(df[col].isin(ems), 2, 0)
    df['fire'] = np.where(df[col].isin(fire), 3, 0)
    
    df['category'] = df['fire'] + df['ems'] + df['police']
    df['category'] = df['category'].replace(0, 1)  #  For some reason the Assault and Battery instances are not picked up
    
    
    defunct = ['police', 'ems', 'fire', 'category']
    
     #  Drop the no longer needed columns

    df = df.drop(columns=defunct)

    return df



In [10]:
def wrangle(df):
    df = reduce_features(df)
    df = drop_null_cols(df)
    df = extract_timestamp_feats(df, 'call_timestamp')
    df = filter_geo_outliers(df)
    df = remove_categoricals(df)
    df = clean_observations(df)
    df = filter_calldescription(df)
    
    return df

In [11]:
df = wrangle(df)

In [13]:
df = df[['latitude', 'longitude', 'calldescription']]

In [14]:
df.head()

,latitude,longitude,calldescription
0,42.365373,-83.197290,UNKNOWN PROBLEM
1,42.353715,-83.062263,INVESTIGATE PERSON
2,42.378907,-83.273309,DISTURBANCE
3,42.369159,-83.263215,PANIC / DURESS ALARM
4,42.425842,-83.067557,ONE DOWN OR OVER THE WHEEL


In [15]:
df.to_csv("scatter_data.csv")